In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 700)
import csv
import json

### 建立df_AD

In [ ]:
with open('json/dept_lst.txt', 'r', encoding='utf8') as filehandle:
    dept_lst = json.load(filehandle)

In [ ]:
# append AD.csv
li = []
for str_dept in dept_lst:
    filepath = r"AD_csv\\"+str_dept+'.csv'    
    df = pd.read_csv(filepath, index_col=None, header=0)
    df['OrganizationalUnit'] = str_dept
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

df_AD = frame[frame.title.notnull()]

In [ ]:
df_AD = df_AD.drop(['samaccountname','description'], axis=1)
df_AD.rename(
    columns = {'department': 'dept_full',
               'OrganizationalUnit': 'dept',
               'displayname': 'name',
               'officephone': 'phone',
              }, inplace = True)

In [ ]:
df_AD[['dept','section']]=df_AD['dept_full'].str.split('-', expand=True)
df_AD.drop(columns=['dept_full'], inplace=True)

### 建立df_tel

In [ ]:
import xlrd

with open('json/tel_xls_file.txt', 'r', encoding='utf8') as filehandle:
    file_name = json.load(filehandle)
print(file_name[0])

wb = xlrd.open_workbook(file_name[0])
table = wb.sheet_by_name(file_name[1]) 

ncols = table.ncols # 28

title_lst = []
name_lst = []
tel_lst = []
comment_lst = []

for i in range(ncols):
    if i % 4 == 0:
        title_lst.extend(table.col_values(i))
    if i % 4 == 1:
        name_lst.extend(table.col_values(i))
    if i % 4 == 2:
        tel_lst.extend(table.col_values(i))
    if i % 4 == 3:
        comment_lst.extend(table.col_values(i))   

In [ ]:
# 移除單名空白
title_lst = [s.replace(' ', '') for s in title_lst]
name_lst = [s.replace(' ', '') for s in name_lst]
comment_lst = [s.replace(' ', '') for s in comment_lst]

In [ ]:
total_len = len(title_lst)
dic_lst = []
for i in range(total_len):
    d = {}
    d['title'] = title_lst[i].strip()
    d['name'] = name_lst[i].strip()
    d['phone'] = tel_lst[i]
    d['comment'] = comment_lst[i].strip()
    dic_lst.append(d)
    
# 由上而下帶入處室單位
with open('json/tel_dept_lst.txt', 'r', encoding='utf8') as filehandle:
    tel_dept_lst = json.load(filehandle)
for i, d in enumerate(dic_lst):
    if i == 0:
        dept = '董事長室'
    if i == 3:
        dept = '總經理室'
    if d['title'] in tel_dept_lst:
        dept = d['title']
    d['dept'] = dept 

In [ ]:
data_lst = [d for d in dic_lst if d['name']!='']

In [ ]:
# 處理科別
with open('json/tel_section_lst.txt', 'r', encoding='utf8') as filehandle:
    tel_section_lst = json.load(filehandle)

import copy
out_put_lst = []
lst = copy.deepcopy(data_lst)

for i in range(len(lst)):
    if lst[i-1]['dept'] == lst[i]['dept']:
        b_new_dept = False
    elif lst[i-1]['dept'] != lst[i]['dept']:
        b_new_dept = True
    
    if lst[i]['comment'] in tel_section_lst:
        section = lst[i]['comment']
    elif b_new_dept:
        section = ''
    lst[i]['section'] = section
    out_put_lst.append(lst[i])

In [ ]:
# 轉成df_tel
df_tel = pd.DataFrame(out_put_lst, columns=['dept', 'name', 'title', 'phone', 'comment', 'section'])
pd.set_option('display.max_rows', df_tel.shape[0]+1)
df_tel['phone'] = df_tel['phone'].astype(np.int64)
# df_tel

In [ ]:
# section加上"科"
df_tel['section'] = np.where(df_tel['section'] == '', df_tel['section'], df_tel['section']+'科')

In [ ]:
# 列出df_tel重複
d = {} # Initializing Dictionary
# counting number of times each word comes up in list of words (in dictionary)
for word in name_lst: 
    d[word] = d.get(word, 0) + 1
    
tel_repeat_name_lst = []
for (key, value) in d.items():
    if value >=2:
        tel_repeat_name_lst.append(key)
tel_repeat_name_lst.remove('')
df_tel[df_tel['name'].isin(tel_repeat_name_lst)] # df_重複名單建立
# 用此df_重複名單建立 tel_repeat_staff.txt

In [ ]:
with open('json/tel_repeat_staff.txt', 'r', encoding='utf8') as filehandle:
    tel_repeat_staff = json.load(filehandle)

In [ ]:
tel_repeat_staff

In [ ]:
# 為了merge暫時drop重複者，之後於df_main額外處理
for lst in tel_repeat_staff:
    df_tel.drop(df_tel[(df_tel.name==lst[0]) & (df_tel.dept==lst[1])].index, inplace=True)
# 特殊處理總務處兼任科長 ->注意hard code 2
df_tel.drop(df_tel[(df_tel.name==tel_repeat_staff[2][0]) & (df_tel.comment==tel_repeat_staff[2][1])].index, inplace=True)

In [ ]:
# 完成建立df_tel

### df_AD合併df_tel

In [ ]:
# 合併
df_merged = pd.merge(df_AD, df_tel, left_on='name', right_on='name', how='left', suffixes=['__AD','__tel'])

In [ ]:
df_merged

### 檢查資料

In [ ]:
# 處室
df_ck = df_merged.copy()
df_ck = df_ck[(df_ck['dept__AD']!=df_ck['dept__tel']) 
          & (~df_ck['dept__tel'].isna())
          ]
df_ck

In [ ]:
# 科別
df_ck = df_merged.copy()
df_ck['section__AD'] = np.where(df_ck['section__AD'].isna(), '', df_ck['section__AD'])
df_ck['section__AD'] = np.where(df_ck['section__AD']=='機電及系統科', '機電科', df_ck['section__AD'])
df_ck = df_ck[(df_ck['section__AD']!=df_ck['section__tel'])
          & (df_ck['dept__AD']!='總管理處')
          & (~df_ck['dept__tel'].isna())
          ]
df_ck

In [ ]:
# 職稱
df_ck = df_merged.copy()
df_ck = df_ck[(df_ck['title__AD']!=df_ck['title__tel']) 
          & (~df_ck['dept__tel'].isna())
          ]
df_ck

In [ ]:
# 電話

In [ ]:
# df_merged = df_merged[(~df_merged['phone__tel'].isna())]

In [ ]:
# df_merged['phone__tel'] = df_merged['phone__tel'].astype(np.int64)

In [ ]:
df_ck = df_merged.copy()
df_ck = df_ck[(df_ck['phone__AD']!=df_ck['phone__tel']) 
          & (~df_ck['dept__tel'].isna())
          ]
df_ck